In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder


## Data Retrieval

Import Telco Dataset

In [2]:
df = pd.read_csv("Telco.csv")

## Data Processing

View the first 5 rows of the dataset

In [3]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


We can see the different data types  present in the data, 1 Float, 2 Integers, 18 Objects. The objects will need to be converted into number values in order to run through the Neural Network.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


There are no null values in the dataframe

In [5]:
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

Drop customer ID as it has no purpose for this model 

In [6]:
df.drop(columns=['customerID'],inplace=True)

Convert all the column names to lowercase to avoid errors in the data wrangling

In [7]:
df.columns = df.columns.str.lower()

Use the lambda function to make all the strings in the dataset lower case so all values are homogenous

In [8]:
df = df.applymap(lambda s: s.lower() if type(s) == str else s)

In [9]:
df

,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,female,0,yes,no,1,no,no phone service,dsl,no,yes,no,no,no,no,month-to-month,yes,electronic check,29.85,29.85,no
1,male,0,no,no,34,yes,no,dsl,yes,no,yes,no,no,no,one year,no,mailed check,56.95,1889.5,no
2,male,0,no,no,2,yes,no,dsl,yes,yes,no,no,no,no,month-to-month,yes,mailed check,53.85,108.15,yes
3,male,0,no,no,45,no,no phone service,dsl,yes,no,yes,yes,no,no,one year,no,bank transfer (automatic),42.30,1840.75,no
4,female,0,no,no,2,yes,no,fiber optic,no,no,no,no,no,no,month-to-month,yes,electronic check,70.70,151.65,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,male,0,yes,yes,24,yes,yes,dsl,yes,no,yes,yes,yes,yes,one year,yes,mailed check,84.80,1990.5,no
7039,female,0,yes,yes,72,yes,yes,fiber optic,no,yes,yes,no,yes,yes,one year,yes,credit card (automatic),103.20,7362.9,no
7040,female,0,yes,yes,11,no,no phone service,dsl,yes,no,no,no,no,no,month-to-month,yes,electronic check,29.60,346.45,no
7041,male,1,yes,no,4,yes,yes,fiber optic,no,no,no,no,no,no,month-to-month,yes,mailed check,74.40,306.6,yes


View all the object data types of the dataframe. As we can see they are all objects.

In [10]:
df.astype(object)

,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,female,0,yes,no,1,no,no phone service,dsl,no,yes,no,no,no,no,month-to-month,yes,electronic check,29.85,29.85,no
1,male,0,no,no,34,yes,no,dsl,yes,no,yes,no,no,no,one year,no,mailed check,56.95,1889.5,no
2,male,0,no,no,2,yes,no,dsl,yes,yes,no,no,no,no,month-to-month,yes,mailed check,53.85,108.15,yes
3,male,0,no,no,45,no,no phone service,dsl,yes,no,yes,yes,no,no,one year,no,bank transfer (automatic),42.3,1840.75,no
4,female,0,no,no,2,yes,no,fiber optic,no,no,no,no,no,no,month-to-month,yes,electronic check,70.7,151.65,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,male,0,yes,yes,24,yes,yes,dsl,yes,no,yes,yes,yes,yes,one year,yes,mailed check,84.8,1990.5,no
7039,female,0,yes,yes,72,yes,yes,fiber optic,no,yes,yes,no,yes,yes,one year,yes,credit card (automatic),103.2,7362.9,no
7040,female,0,yes,yes,11,no,no phone service,dsl,yes,no,no,no,no,no,month-to-month,yes,electronic check,29.6,346.45,no
7041,male,1,yes,no,4,yes,yes,fiber optic,no,no,no,no,no,no,month-to-month,yes,mailed check,74.4,306.6,yes


## Data Preprocessing

We must use pandas df.replace function in order to replace all of the yes and no values to numbers in order for the Neural network model to run correctly

In [11]:
df.replace(['yes'], 1, inplace=True)
df.replace(['no'], 0, inplace=True)
df.replace(['no internet service'], 0,inplace=True)
df.replace(['no phone service'],0,inplace=True)
df['internetservice'].replace(["dsl"], 2, inplace=True)
df['internetservice'].replace(["fiber optic"], 1, inplace=True)

In [12]:
df

,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,female,0,1,0,1,0,0,2,0,1,0,0,0,0,month-to-month,1,electronic check,29.85,29.85,0
1,male,0,0,0,34,1,0,2,1,0,1,0,0,0,one year,0,mailed check,56.95,1889.5,0
2,male,0,0,0,2,1,0,2,1,1,0,0,0,0,month-to-month,1,mailed check,53.85,108.15,1
3,male,0,0,0,45,0,0,2,1,0,1,1,0,0,one year,0,bank transfer (automatic),42.30,1840.75,0
4,female,0,0,0,2,1,0,1,0,0,0,0,0,0,month-to-month,1,electronic check,70.70,151.65,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,male,0,1,1,24,1,1,2,1,0,1,1,1,1,one year,1,mailed check,84.80,1990.5,0
7039,female,0,1,1,72,1,1,1,0,1,1,0,1,1,one year,1,credit card (automatic),103.20,7362.9,0
7040,female,0,1,1,11,0,0,2,1,0,0,0,0,0,month-to-month,1,electronic check,29.60,346.45,0
7041,male,1,1,0,4,1,1,1,0,0,0,0,0,0,month-to-month,1,mailed check,74.40,306.6,1


Check if there is any whitespace in the string values of the dataframe. We Can see that there is a few.

In [13]:
np.where(df == " ")

(array([ 488,  753,  936, 1082, 1340, 3331, 3826, 4380, 5218, 6670, 6754],
       dtype=int64),
 array([18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18], dtype=int64))

Using indexing we can find an example of the whitespace present in the data 

In [14]:
df.totalcharges.iloc[485:490]

485    6130.85
486       1415
487    6201.95
488           
489      74.35
Name: totalcharges, dtype: object

Replace the whitespace values with 0

In [15]:
df.replace(" ",0, inplace=True)

Initialise the label encoder from SkLearn.preprocessing

In [16]:
label = LabelEncoder()

Create an array of the dataframe and take all features of the data except for "churn" and store it in "X".
</br></br>
Store the Churn feature as our target variable in "Y"

In [17]:
X = df.iloc[:,:-1].values
Y = df.iloc[:,-1:].values
X.shape

(7043, 19)

Use the label encoder to encode and transform the remaining values into numeric form and assign an integer value to each

In [18]:
X[:,7] = label.fit_transform(X[:,7])
X[:,-5] = label.fit_transform(X[:,-5])
X[:,-4] = label.fit_transform(X[:,-4])
X[:,-3] = label.fit_transform(X[:,-3])
X[:,0] = label.fit_transform(X[:,0])

In [19]:
X

array([[0, 0, 1, ..., 2, 29.85, '29.85'],
       [1, 0, 0, ..., 3, 56.95, '1889.5'],
       [1, 0, 0, ..., 3, 53.85, '108.15'],
       ...,
       [0, 0, 1, ..., 2, 29.6, '346.45'],
       [1, 1, 1, ..., 3, 74.4, '306.6'],
       [1, 0, 0, ..., 0, 105.65, '6844.5']], dtype=object)

Initialise the ColumnTransformer from SkLearn.compose to transform the 19 features present in the "X" variable into a single feature space

In [20]:
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder='passthrough')
X = np.array(ct.fit_transform(X))

Split the data using Train Test Split with a 70/30 split

In [21]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,train_size=0.3)

In [22]:
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((2112, 20), (4931, 20), (2112, 1), (4931, 1))

Scale the data using standard Scaler to convert the data to the same scale.

In [23]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## ANN

Initialise the Sequential model of the Neural Network 

In [29]:
ann = tf.keras.models.Sequential()

For this Neural Network we will use the Rectified Linear Unit activation function and the Sigmoid Activation Function as we have a binary target variable.

In [30]:
ann.add(tf.keras.layers.Dense(units= 8, activation='relu'))
ann.add(tf.keras.layers.Dense(units= 8, activation='relu'))
ann.add(tf.keras.layers.Dense(units= 1, activation='sigmoid'))

Using a Binary Cross Entropy loss function we will measure the accuracy of our model 

In [34]:
ann.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
ann.fit(X_train,Y_train,batch_size=32,epochs=200)

Epoch 1/200
66/66 [==============================] - 0s 924us/step - loss: 0.3613 - accuracy: 0.8385
Epoch 2/200
66/66 [==============================] - 0s 939us/step - loss: 0.3598 - accuracy: 0.8381
Epoch 3/200
66/66 [==============================] - 0s 924us/step - loss: 0.3602 - accuracy: 0.8381
Epoch 4/200
66/66 [==============================] - 0s 924us/step - loss: 0.3602 - accuracy: 0.8357
Epoch 5/200
66/66 [==============================] - 0s 955us/step - loss: 0.3597 - accuracy: 0.8371
Epoch 6/200
66/66 [==============================] - 0s 909us/step - loss: 0.3591 - accuracy: 0.8376
Epoch 7/200
66/66 [==============================] - 0s 924us/step - loss: 0.3590 - accuracy: 0.8381
Epoch 8/200
66/66 [==============================] - 0s 955us/step - loss: 0.3587 - accuracy: 0.8395
Epoch 9/200
66/66 [==============================] - 0s 939us/step - loss: 0.3588 - accuracy: 0.8376
Epoch 10/200
66/66 [==============================] - 0s 970us/step - loss: 0.3597 - accura

66/66 [==============================] - 0s 939us/step - loss: 0.3531 - accuracy: 0.8381
Epoch 82/200
66/66 [==============================] - 0s 939us/step - loss: 0.3526 - accuracy: 0.8385
Epoch 83/200
66/66 [==============================] - 0s 955us/step - loss: 0.3520 - accuracy: 0.8385
Epoch 84/200
66/66 [==============================] - 0s 939us/step - loss: 0.3518 - accuracy: 0.8404
Epoch 85/200
66/66 [==============================] - 0s 924us/step - loss: 0.3522 - accuracy: 0.8414
Epoch 86/200
66/66 [==============================] - 0s 955us/step - loss: 0.3528 - accuracy: 0.8395
Epoch 87/200
66/66 [==============================] - 0s 924us/step - loss: 0.3519 - accuracy: 0.8385
Epoch 88/200
66/66 [==============================] - 0s 909us/step - loss: 0.3520 - accuracy: 0.8376
Epoch 89/200
66/66 [==============================] - 0s 909us/step - loss: 0.3519 - accuracy: 0.8404
Epoch 90/200
66/66 [==============================] - 0s 909us/step - loss: 0.3520 - accuracy: 

66/66 [==============================] - 0s 909us/step - loss: 0.3463 - accuracy: 0.8456
Epoch 161/200
66/66 [==============================] - 0s 893us/step - loss: 0.3469 - accuracy: 0.8456
Epoch 162/200
66/66 [==============================] - 0s 924us/step - loss: 0.3456 - accuracy: 0.8433
Epoch 163/200
66/66 [==============================] - 0s 939us/step - loss: 0.3462 - accuracy: 0.8480
Epoch 164/200
66/66 [==============================] - 0s 955us/step - loss: 0.3460 - accuracy: 0.8447
Epoch 165/200
66/66 [==============================] - 0s 924us/step - loss: 0.3457 - accuracy: 0.8452
Epoch 166/200
66/66 [==============================] - 0s 909us/step - loss: 0.3473 - accuracy: 0.8452
Epoch 167/200
66/66 [==============================] - 0s 939us/step - loss: 0.3458 - accuracy: 0.8456
Epoch 168/200
66/66 [==============================] - 0s 970us/step - loss: 0.3461 - accuracy: 0.8414
Epoch 169/200
66/66 [==============================] - 0s 970us/step - loss: 0.3454 - a

Using this neural network we can predict with 78% the Customer churn from Telco's business

In [36]:
loss, accuracy = ann.evaluate(X_test, Y_test)

155/155 [==============================] - 0s 819us/step - loss: 0.4814 - accuracy: 0.7789
